# Pandas und Funktionen

**Inhalt:** Selbst definierte oder importierte Funktionen in Pandas anwenden

**Nötige Skills:** Einführung in Pandas

**Lernziele:**
- Review: Mehrere Codezeilen zu Funktionen zusammenfassen
- Funktionen auf bestimmte Bestandteile von Dataframes ausführen (apply)
- Funkionen on-the-fly definiteren und anwenden (lambda)

## Das Beispiel

Eine kleine Datenbank der besten alltime-Singles aus der Schweizer Hitparade und den Songtexten dazu.

Wir betreiben etwas Textanalyse und experimentieren mit einem Ranking.

Quellen:
- https://hitparade.ch/charts/best/singles
- https://www.songtexte.com/

Das Scrape-File dazu findet sich hier: `dataprojects/Songtexte/scraper.ipynb`

Das Daten-File hier: `dataprojects/Songtexte/charts_mit_texten.csv`

## Vorbereitung

In [ ]:
import pandas as pd

In [ ]:
import numpy as np

In [ ]:
pd.set_option('display.max_colwidth', 2000)

## Daten laden

In [ ]:
df = pd.read_csv('dataprojects/Songtexte/charts_mit_texten.csv')

In [ ]:
df.head()

Achtung, ein paar Songs haben keinen Text

In [ ]:
df['Songtext'] = df['Songtext'].replace(np.nan, "")

## Datenbeispiel

Hier der Text des ersten Songs in unserer Datenbank:

In [ ]:
df[df['Rang'] == 1]['Songtext']

In [ ]:
print(df[df['Rang'] == 1]['Songtext'][0])

## Analyse über ein Datenfeld

Was könnten wir an diesem Songtext auswerten? Überlegen Sie sich Möglichkeiten für eine simple Textanalyse.

In [ ]:
# Beispiele:

# Anzahl Zeilen
# Anzahl Wörter
# Durchschnittliche Länge der Wörter
# Anzahl Strophen und Refrains
# Vokale vs. Konsonanten
# etc.

Wir machen es uns einfach, und werten die Anzahl Zeilen aus. Dazu schreiben wir eine Funktion:
- Sie erhält als Input einen String
- Sie liefert als Output eine Zahl
- Achtung: Es hat auch doppelte Zeilenumbrüche drin

### Funktion schreiben

In [ ]:
def zeilenzahl(text):

    # Wir zählen zuerst alle Zeilenumbrüche
    zeilenumbrueche = text.count("\n")
    
    # Anschliessend zählen wir die doppelten Zeilenumbrüche...
    doppelte_zeilenumbrueche = text.count("\n\n")
    
    # ... und subtrahieren diesen Wert, um die korrekte Zeilenzahl herauszufinden
    zeilen = zeilenumbrueche - doppelte_zeilenumbrueche
    
    return zeilen

### Funktion testen

In [ ]:
test_text = '''
Dies ist ein Text mit drei Zeilen.
Zwischen der zweiten und dritten Zeile hat es einen doppelten Umbruch.

Gibt unsere Funktion den richtigen Wert aus?'''

In [ ]:
test_text

In [ ]:
print(test_text)

In [ ]:
zeilenzahl(test_text)

### Funktion anwenden

Damit wir unsere Zählfunkion auf alle Songtexte in der Datenbank anwenden können, brauchen wir `apply()`.

Pandas wendet dann unsere Funktion `zeilenzahl()` auf jedes der Elemente in der Series `df['Songtext']` an.

In [ ]:
df['Songtext'].apply(zeilenzahl)

Notice: Wir müssen in die Klammern der Funktion `zeilenzahl` hier kein Argument angeben!

Warum brauchen wir `apply()`? Weil dieser Befehl hier nicht funktioniert

In [ ]:
# zeilenzahl(df['Songtext'])

### Ergebnis speichern

Das Resultat können wir auch als separate Spalte speichern:

In [ ]:
df['Zeilenzahl'] = df['Songtext'].apply(zeilenzahl)

In [ ]:
df.head()

### Analyse

Zum Beispiel die Fragestellung: Wie hat sich die Länge der Songtexte über die Zeit verändert?

In [ ]:
df['Eintritt'] = pd.to_datetime(df['Eintritt'])

In [ ]:
df_temp = df[(df['Eintritt'].dt.year >= 2005) & (df['Zeilenzahl'] != 0)]
df_temp.plot(x='Eintritt', y='Zeilenzahl', figsize=(10,6), title="Hat sich die Länge der Songtexte verändert?")

## Analyse über mehrere Datenfelder

Pandas `apply()` kann nicht nur verwendet werden, um die Elemente einer Series zu verarbeiten, sondern auch um eine Funktion zeilenweise auf ein Dataframe anzuwenden. Das ist nötig, sobald wir Inputs aus mehreren Spalten brauchen.

Zum Beispiel die Frage: Wie oft kommt der Songtitel im Songtext vor?

### Funktion schreiben

Wir brauchen eine Funktion:
- mit einem Input: Die ganze Tabellenzeile *als Series bzw. Dictionary*
- mit einem Output: Anzazhl Repetitionen
- Achtung auf Sonderzeichen im Titel!

In [ ]:
def wiederholungen(row):
    
    # Zuerst erstellen wir eine Liste von Sonderzeichen
    sonderzeichen = ['!', '"', '&', "'", '(', ')', '–', '.', '?']
    
    # Wir entfernen diese Sonderzeichen aus dem Titel und Text
    bereinigter_titel = row['Titel']
    bereinigter_text = row['Songtext']

    for zeichen in sonderzeichen:
        bereinigter_titel = bereinigter_titel.replace(zeichen, "")
        bereinigter_text = bereinigter_text.replace(zeichen, "")
        
    # Jetzt können wir die Zählung starten

    wiederholungen = bereinigter_text.count(bereinigter_titel)
    
    return wiederholungen

### Funktion Testen

In [ ]:
titel = "Ho! Ho! Ho!"

In [ ]:
songtext = '''
Dies ist das Lied vom Weihnachtsmann
Ho Ho Ho
Der Weihnachtsmann kommt an Weihnachten
Ho Ho Ho
Kommt der Weihnachtsmann an Weihnachten?
Ho? Ho? Ho?
Ja er kommt an Weihnachten
Ho Ho Ho
'''

In [ ]:
row = {
    'Titel': titel,
    'Songtext': songtext
}

In [ ]:
wiederholungen(row)

### Funktion anwenden

Das lässige ist: Die Funktion weiss jetzt automatisch, welche Spalte es sich rauspicken muss.

Mit `apply()` wenden wir nun die Funktio `wiederholungen()` auf jede Zeile in unserem `df` an.

Achtung auf Parameter `axis=1`!

In [ ]:
df.apply(wiederholungen, axis=1)

Das Ergebnis können wir wiederum speichern...

In [ ]:
df['Wiederholungen'] = df.apply(wiederholungen, axis=1)

Wie sieht das aus?

In [ ]:
df['Wiederholungen'].value_counts()

In [ ]:
df.sort_values('Wiederholungen', ascending=False).head(2)

Sind Songs mit vielen Titel-Wiederholungen besonders erfolgreich?

In [ ]:
df.groupby('Wiederholungen')['Punkte'].mean().plot()

## Analyse über mehrere Datenfelder (mit Lambda)

Manchmal müssen wir eine Funktion auch auf andere Weise aufrufen. ZB dann, wenn sie von extern importiert wurde oder nicht eine Series als Input benötigt, sondern einzelne Variablen.

Das wäre zB hier der Fall. Statt so...

In [ ]:
def wiederholungen(row):
    ...

... sieht das nun so aus:

In [ ]:
def wiederholungen(titel, songtext):
    
    # Zuerst erstellen wir eine Liste von Sonderzeichen
    sonderzeichen = ['!', '"', '&', "'", '(', ')', '–', '.', '?']
    
    # Wir entfernen diese Sonderzeichen aus dem Titel und Text
    bereinigter_titel = titel # <= VORHER: row['Titel']
    bereinigter_text = songtext # <= VORHER: row['Songtext']

    for zeichen in sonderzeichen:
        bereinigter_titel = bereinigter_titel.replace(zeichen, "")
        bereinigter_text = bereinigter_text.replace(zeichen, "")
        
    # Jetzt können wir die Zählung starten

    wiederholungen = bereinigter_text.count(bereinigter_titel)
    
    return wiederholungen

Um diese Funktion zu benutzen, müssen wir mit dem Keyword `lambda` arbeiten.

In [ ]:
df.apply(lambda row: wiederholungen(row['Titel'], row['Songtext']), axis=1)

Lambda-Funktionen sehen auf den ersten Blick kompliziert aus, sind aber keine Hexerei.

Erklärung siehe zB hier: https://www.programiz.com/python-programming/anonymous-function

Im wesentlichen sind es Funktionen, die on-the-fly definiert und ausgeführt werden, aber ohne dass sie einen Namen erhalten.

In [ ]:
lambda x: x + 1

In [ ]:
(lambda x: x + 1)(5)